# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [2]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('C:/Users/Test/production/05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [3]:
from sklearn.model_selection import train_test_split

# Separate features and target
X = adult_dt.drop('income', axis=1)
Y = adult_dt['income']

# Train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)



## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

Random state is necessary to ensure the train-test split is reproducible. We need to ensure we get the same split every time we run the code, it ensures consistency. 

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn import set_config

# Define feature names 
numeric_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 
                        'race', 'sex', 'native-country']

# Numerical pipeline with steps
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

# Categorical pipeline with steps 
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop='if_binary', handle_unknown='ignore'))
])

# Combine pipelines into ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Wrap ColumnTransformer 
full_pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Display the Pipeline <-- was unable to trigger the visual of the pipeline,
# so I used print to display instead 
print(full_pipeline)

# After confirming the visual, proceed to fit and transform the data
full_pipeline.fit(X_train)
X_train_transformed = full_pipeline.transform(X_train)

# Display the shape of the transformed data to verify it 
print("Transformed shape:", X_train_transformed.shape)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',


## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

# Create model pipeline with preprocessing and classifier 
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor), 
    ('classifier', RandomForestClassifier())
])

# Fit pipeline on the training data
model_pipeline.fit(X_train, Y_train)

# Sample prediction to confirm 
sample_predictions = model_pipeline.predict(X_train[:5])
print("Sample Predictions:", sample_predictions)


Sample Predictions: [0 1 0 0 0]


# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [10]:
from sklearn.model_selection import cross_validate
import pandas as pd

# Define the scoring metrics for cv 
scoring_metrics = {
    'neg_log_loss': 'neg_log_loss',
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
}

# Perform cv 
cv_results = cross_validate(
    model_pipeline, 
    X_train, 
    Y_train, 
    cv=5, 
    scoring=scoring_metrics, 
    return_train_score=True
)



c:\Users\Test\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Test\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [11]:
# Convert to df, sort by test_neg_log_loss 
cv_results_df = pd.DataFrame(cv_results)
cv_results_df = cv_results_df.sort_values(by='test_neg_log_loss')

# Display df sorted by test negative log loss
print("Cross-Validation Results:")
print(cv_results_df)


Cross-Validation Results:
    fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
4  15.943826    0.636475          -0.384362           -0.081254      0.902110   
2  14.731450    0.330858          -0.379601           -0.081640      0.901302   
3  16.290673    0.449172          -0.367867           -0.081929      0.906622   
0  12.646333    0.370782          -0.364281           -0.081762      0.902964   
1  16.908556    0.490830          -0.349989           -0.081247      0.903690   

   train_roc_auc  test_accuracy  train_accuracy  test_balanced_accuracy  \
4            1.0       0.855419        1.000000                0.776578   
2            1.0       0.853664        0.999945                0.775419   
3            1.0       0.860026        1.000000                0.784849   
0            1.0       0.848212        1.000000                0.771662   
1            1.0       0.846238        0.999945                0.767901   

   train_balanced_accuracy  
4      

Calculate the mean of each metric. 

In [12]:
# Calculate the mean of each metric
mean_metrics = cv_results_df.mean()
print("\nMean of each metric:")
print(mean_metrics)


Mean of each metric:
fit_time                   15.304168
score_time                  0.455624
test_neg_log_loss          -0.369220
train_neg_log_loss         -0.081567
test_roc_auc                0.903337
train_roc_auc               1.000000
test_accuracy               0.852712
train_accuracy              0.999978
test_balanced_accuracy      0.775282
train_balanced_accuracy     0.999955
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [13]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Generate predictions and prediction probabilities on the test set
Y_pred = model_pipeline.predict(X_test)
Y_pred_proba = model_pipeline.predict_proba(X_test)

# Calculate the test metrics
test_metrics = {
    'log_loss': log_loss(Y_test, Y_pred_proba),
    'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),  # Only pass the probability for the positive class
    'accuracy': accuracy_score(Y_test, Y_pred),
    'balanced_accuracy': balanced_accuracy_score(Y_test, Y_pred)
}

# Display the test metrics as a dictionary
print("Test Metrics:")
for metric, value in test_metrics.items():
    print(f"{metric}: {value:.4f}")


Test Metrics:
log_loss: 0.3683
roc_auc: 0.9010
accuracy: 0.8542
balanced_accuracy: 0.7753


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

Recoding immediately upon loading the data is (I would consider) best practice.  It's best to do immediately to ensure compatibility, consistency, and it can make working with the data more efficient. 

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.